<a href="https://colab.research.google.com/github/goldwyns/Epilepsy-Classification-Using-Novel-Feature-Set/blob/main/Feature_Selection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spectrum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 19.7 MB/s eta 0:00:00
  Created wheel for spectrum: filename=spectrum-0.9.0-cp311-cp311-linux_x86_64.whl size=236745 sha256=767e2df6622508c9f96cfd3011fcd8854cd5de4df1dd97f537403d9f6adc81d3
  Stored in directory: /root/.cache/pip/wheels/c0/9c/de/eb558fbd03ea1540d3c908f23681f57f9d9e8c2a5cd08d6f42
Successfully built spectrum


In [ ]:
import os
import numpy as np
from scipy.signal import detrend, butter, filtfilt
from scipy.stats import skew, kurtosis
from scipy.linalg import toeplitz
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score, matthews_corrcoef, roc_auc_score

# Base paths with labels
base_paths = {
    's': "/content/drive/MyDrive/Database/Bonn Univ Dataset/s/S/",
    'z': "/content/drive/MyDrive/Database/Bonn Univ Dataset/z/Z/",
    'o': "/content/drive/MyDrive/Database/Bonn Univ Dataset/o/O/",
    'f': "/content/drive/MyDrive/Database/Bonn Univ Dataset/f/F/",
    'n': "/content/drive/MyDrive/Database/Bonn Univ Dataset/n/N/"
}

# Load EEG Data
def load_eeg_data(base_path):
    print(f"Loading EEG data from {base_path}")
    eeg_data = []
    file_count = 0
    for file_name in os.listdir(base_path):
        if file_name.endswith(".txt"):
            file_count += 1
            file_path = os.path.join(base_path, file_name)
            data = np.loadtxt(file_path)
            if len(data) > 4097:
                print(f"File {file_name} has more than 4097 data points: {len(data)}")
            eeg_data.append(data)
    print(f"Number of files loaded: {file_count}")
    if len(eeg_data) == 0:
        raise ValueError(f"No data found in directory: {base_path}")
    return eeg_data

# Preprocess EEG signal
def preprocess_signal(signal, lowcut=0.5, highcut=60.0, fs=173.61):
    # print("Preprocessing EEG signal")
    signal_detrended = detrend(signal)
    nyquist = 0.5 * fs
    if lowcut >= highcut:
        raise ValueError("Lowcut frequency must be less than highcut frequency.")
    if lowcut <= 0 or highcut >= nyquist:
        raise ValueError(f"Lowcut must be > 0 and highcut must be < Nyquist frequency ({nyquist} Hz).")
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    signal_filtered = filtfilt(b, a, signal_detrended)
    scaler = StandardScaler()
    signal_normalized = scaler.fit_transform(signal_filtered.reshape(-1, 1)).flatten()
    return signal_normalized

# Segment and preprocess data before splitting
def segment_signal(signal, fs=173.61, window_duration=2.0, overlap_duration=1.0):
    window_size = int(window_duration * fs)
    overlap = int(overlap_duration * fs)
    step = window_size - overlap
    segments = []
    for start in range(0, len(signal) - window_size + 1, step):
        segment = signal[start:start + window_size]
        segments.append(segment)
    print(f"Number of segments created: {len(segments)}")
    return segments

def segment_and_preprocess(data, label, fs=173.61, window_duration=2.0, overlap_duration=1.0):
    print(f"Segmenting and preprocessing data for label: {label}")
    segments, labels = [], []
    for single_data in data:
        segmented_signals = segment_signal(single_data, fs, window_duration, overlap_duration)
        for segment in segmented_signals:
            preprocessed_segment = preprocess_signal(segment, lowcut=0.5, highcut=60.0, fs=fs)
            segments.append(preprocessed_segment)
            labels.append(label)
    return segments, labels

# Feature extraction functions (using previously defined functions)
def extract_normal_features(segment):
    return [np.mean(segment), np.var(segment), skew(segment), kurtosis(segment), np.median(segment)]

def extract_lpc_features(segment, order=10):
    autocorr = np.correlate(segment, segment, mode='full')
    autocorr = autocorr[autocorr.size // 2:]
    R = toeplitz(autocorr[:order])
    r = autocorr[1:order + 1]
    a = np.linalg.solve(R, -r)
    a = np.hstack([1, a])
    return [np.mean(a), np.var(a), skew(a), kurtosis(a), np.median(a)]

def extract_cepstral_features(segment, num_coefficients=13):
    spectrum = np.fft.fft(segment)
    log_spectrum = np.log(np.abs(spectrum) + 1e-8)
    cepstrum = np.fft.ifft(log_spectrum).real
    coefficients = cepstrum[:num_coefficients]
    return [np.mean(coefficients), np.var(coefficients), skew(coefficients), kurtosis(coefficients), np.median(coefficients)]

def extract_lattice_features(segment, order=10):
    from scipy.signal import lfilter
    from numpy.linalg import lstsq
    x = segment
    y = lfilter([1], [1] + [-0.9], x)
    a, res, rank, s = lstsq(toeplitz(y[:-1], x[:1]), y[1:], rcond=None)  # Updated with rcond=None
    return [np.mean(a), np.var(a), skew(a), kurtosis(a), np.median(a)]

def extract_inverse_filter_features(segment, order=10):
    from statsmodels.tsa.ar_model import AutoReg
    model = AutoReg(segment, lags=order, old_names=False)
    model_fit = model.fit()
    coefficients = model_fit.params
    return [np.mean(coefficients), np.var(coefficients), skew(coefficients), kurtosis(coefficients), np.median(coefficients)]

def extract_spectral_features(segment, order=10):
    from spectrum import aryule
    _, rho, ref_coeffs = aryule(segment, order)
    return [np.mean(ref_coeffs), np.var(ref_coeffs), skew(ref_coeffs), kurtosis(ref_coeffs), np.median(ref_coeffs)]

def extract_features_by_type(segment, feature_type):
    if feature_type == 'normal':
        return extract_normal_features(segment)
    elif feature_type == 'lpc':
        return extract_lpc_features(segment)
    elif feature_type == 'cepstral':
        return extract_cepstral_features(segment)
    elif feature_type == 'lattice':
        return extract_lattice_features(segment)
    elif feature_type == 'inverse_filter':
        return extract_inverse_filter_features(segment)
    elif feature_type == 'spectral':
        return extract_spectral_features(segment)
    else:
        raise ValueError("Invalid feature type")

def extract_features_from_segments(segments, feature_type):
    features = [extract_features_by_type(segment, feature_type) for segment in segments]
    return np.array(features)


**2. Neural Network Model Definitions**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Bidirectional, Flatten, Dropout, Input, Layer, MultiHeadAttention, LayerNormalization, Embedding

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

def create_bi_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(64, return_sequences=True))
    model.add(GRU(64))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model



TransformerBlock Implementation

In [ ]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def create_transformer_model(input_shape, embed_dim=32, num_heads=2, ff_dim=32):
    inputs = tf.keras.Input(shape=input_shape)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(inputs)
    x = Flatten()(x)
    x = Dense(32, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


Capsule Network Implementation


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.num_capsules * self.dim_capsules),
                                      initializer='glorot_uniform', trainable=True)

    def call(self, inputs):
        # Ensure inputs have a valid shape
        if inputs.shape[1] is None or inputs.shape[2] is None:
            raise ValueError("Input tensor must have defined shape for all dimensions except batch size.")

        inputs_expand = tf.expand_dims(inputs, axis=2)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules * self.dim_capsules, 1])
        inputs_hat = tf.map_fn(lambda x: tf.matmul(self.kernel, x), elems=inputs_tiled)
        inputs_hat_reshaped = tf.reshape(inputs_hat, [-1, inputs.shape[1], self.num_capsules, self.dim_capsules])
        b = tf.zeros(shape=[inputs.shape[0], inputs.shape[1], self.num_capsules])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=-1)
            s = tf.reduce_sum(tf.multiply(c, inputs_hat_reshaped), axis=1)
            v = self.squash(s)
            if i < self.routings - 1:
                b = b + tf.reduce_sum(tf.multiply(inputs_hat_reshaped, tf.expand_dims(v, axis=1)), axis=-1)

        return v

    def squash(self, s):
        s_norm = tf.norm(s, axis=-1, keepdims=True)
        return (s_norm / (1 + s_norm ** 2)) * (s / s_norm)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.num_capsules, self.dim_capsules)

def create_capsule_model(input_shape, num_capsules=10, dim_capsules=16, routings=3):
    inputs = tf.keras.Input(shape=input_shape)
    conv_layer = layers.Conv1D(256, kernel_size=3, strides=1, padding='valid', activation='relu')(inputs)
    primary_capsules = layers.Conv1D(256, kernel_size=3, strides=2, padding='valid')(conv_layer)
    primary_capsules = layers.Reshape(target_shape=[-1, dim_capsules])(primary_capsules)
    capsule_layer = CapsuleLayer(num_capsules=num_capsules, dim_capsules=dim_capsules, routings=routings)(primary_capsules)
    capsule_layer = layers.Flatten()(capsule_layer)
    capsule_layer = layers.Dense(32, activation='relu')(capsule_layer)
    outputs = layers.Dense(1, activation='sigmoid')(capsule_layer)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


3. Training and Evaluation


In [ ]:
from keras.preprocessing.sequence import pad_sequences
def run_binary_classification_nn(class1, class2):
    all_segments, all_labels = [], []
    for label in [class1, class2]:
        path = base_paths[label]
        eeg_data = load_eeg_data(path)
        segments, labels = segment_and_preprocess(eeg_data, label)
        all_segments.extend(segments)
        all_labels.extend(labels)

    label_mapping = {class1: 0, class2: 1}
    all_labels = [label_mapping[label] for label in all_labels]

    # Define feature types
    feature_types = ['normal', 'lpc', 'cepstral', 'inverse_filter', 'spectral']

    # Initialize results
    all_results = {}

    for feature_type in feature_types:
        # Extract features from segments
        features = extract_features_from_segments(all_segments, feature_type)

        # Handle missing values using SimpleImputer
        imputer = SimpleImputer(strategy='mean')
        features = imputer.fit_transform(features)

        # Pad sequences if necessary
        X = pad_sequences(features, maxlen=10, dtype='float32', padding='post', truncating='post')
        X = X.reshape(len(X), X.shape[1], 1)

        # Convert all_labels to a NumPy array
        all_labels = np.array(all_labels)

        X_train, X_test, y_train, y_test = train_test_split(X, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

        input_shape = X_train.shape[1:]

        # Define the neural network models
        models = {
            'LSTM': create_lstm_model(input_shape),
            'Bi-LSTM': create_bi_lstm_model(input_shape),
            'GRU': create_gru_model(input_shape),
            'Transformer': create_transformer_model(input_shape),
        }

        results = {}
        for name, model in models.items():
            print(f"Training {name} model with {feature_type} features")
            model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
            y_pred = (model.predict(X_test) > 0.5).astype("int32").flatten()
            accuracy = accuracy_score(y_test, y_pred)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            f1 = f1_score(y_test, y_pred)
            mcc = matthews_corrcoef(y_test, y_pred)
            y_prob = model.predict(X_test).flatten()
            auc = roc_auc_score(y_test, y_prob)
            cm = confusion_matrix(y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()

            results[name] = {
                'Accuracy': accuracy,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'F1 Score': f1,
                'Matthews Coefficient': mcc,
                'AUC': auc,
                'True Positive': tp,
                'True Negative': tn,
                'False Positive': fp,
                'False Negative': fn
            }

        all_results[feature_type] = results

    return all_results


Complete Display Results Function


In [ ]:
def display_results(all_results, class1, class2):
    print(f"\nClassification Results for {class1} vs {class2}:\n")
    for feature_type, results in all_results.items():
        print(f"Feature Type: {feature_type}")
        for name, metrics in results.items():
            print(f"Classifier: {name}")
            print(f"Accuracy: {metrics['Accuracy']:.4f}")
            print(f"Sensitivity (Recall for Positive Class): {metrics['Sensitivity']:.4f}")
            print(f"Specificity (Recall for Negative Class): {metrics['Specificity']:.4f}")
            print(f"F1 Score: {metrics['F1 Score']:.4f}")
            print(f"Matthews Correlation Coefficient: {metrics['Matthews Coefficient']:.4f}")
            print(f"AUC: {metrics['AUC']:.4f}")
            print(f"True Positives: {metrics['True Positive']}")
            print(f"True Negatives: {metrics['True Negative']}")
            print(f"False Positives: {metrics['False Positive']}")
            print(f"False Negatives: {metrics['False Negative']}")
            print("-" * 50)

if __name__ == "__main__":
    # Define the classes you want to compare
    class1 = 's'
    class2 = 'z'

    # Run the binary classification with neural network models
    all_results = run_binary_classification_nn(class1, class2)

    # Display the results
    display_results(all_results, class1, class2)


Loading EEG data from /content/drive/MyDrive/Database/Bonn Univ Dataset/s/S/
Number of files loaded: 100
Segmenting and preprocessing data for label: s
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of s

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.impute import SimpleImputer

# Function to run unsupervised learning algorithms
def run_unsupervised_learning(class1, class2):
    all_segments, all_labels = [], []
    for label in [class1, class2]:
        path = base_paths[label]
        eeg_data = load_eeg_data(path)
        segments, labels = segment_and_preprocess(eeg_data, label)
        all_segments.extend(segments)
        all_labels.extend(labels)

    label_mapping = {class1: 0, class2: 1}
    all_labels = [label_mapping[label] for label in all_labels]

    # Define feature types
    feature_types = ['normal', 'lpc', 'cepstral', 'inverse_filter', 'spectral']

    # Initialize results
    all_results = {}

    for feature_type in feature_types:
        # Extract features from segments
        features = extract_features_from_segments(all_segments, feature_type)

        # Handle missing values using SimpleImputer
        imputer = SimpleImputer(strategy='mean')
        features = imputer.fit_transform(features)

        # Standardize features
        scaler = StandardScaler()
        features = scaler.fit_transform(features)

        # Initialize unsupervised learning models
        models = {
            'K-Means': KMeans(n_clusters=2),
            'Hierarchical Clustering': AgglomerativeClustering(n_clusters=2),
            'DBSCAN': DBSCAN(),
            'GMM': GaussianMixture(n_components=2)
        }

        results = {}
        for name, model in models.items():
            print(f"Running {name} on {feature_type} features")
            model.fit(features)
            if hasattr(model, 'labels_'):
                labels = model.labels_
            else:
                labels = model.predict(features)
            silhouette_avg = silhouette_score(features, labels)
            results[name] = {
                'Silhouette Score': silhouette_avg
            }

        all_results[feature_type] = results

    return all_results

# New function to display results
def display_unsupervised_learning(all_results, class1, class2):
    for feature_type, results in all_results.items():
        print(f"\nResults for {feature_type} features:")
        for model_name, metrics in results.items():
            print(f"{model_name}:")
            for metric, value in metrics.items():
                print(f"  {metric}: {value}")

if __name__ == "__main__":
    # Define the classes you want to compare
    class1 = 's'
    class2 = 'z'

    # Run the unsupervised learning algorithms
    all_results = run_unsupervised_learning(class1, class2)

    # Display the results using the new function
    display_unsupervised_learning(all_results, class1, class2)


Loading EEG data from /content/drive/MyDrive/Database/Bonn Univ Dataset/s/S/
Number of files loaded: 100
Segmenting and preprocessing data for label: s
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of segments created: 22
Number of s